In [1]:
# To reload modules easily
%load_ext autoreload
%autoreload 2

In [4]:
from embedding.word2vec_model import Word2VecModel
from data_load.embedding_loader import EmbeddingDataset, EmbeddingDataLoader
# from model.rnn import BasicLSTM
# from train.base_trainer import DataLoaderTrainer
# from generate.base_generator import LineEmbeddingGenerator

In [8]:
preprocessed_data = 'preprocess/output/result.txt'

현재 size=100, window=2, min_count=10, workers=4, sg=1, sample=1e-5,

In [9]:
embeddingSize = 200

In [25]:
# embedding = Word2VecModel(size = embeddingSize)
embedding = Word2VecModel()
embedding.build_embedding(preprocessed_data, 'output', refresh=True)

In [26]:
bamboo2vec = embedding.model

# T-SNE

In [27]:
from sklearn.manifold import TSNE

In [28]:
import pandas as pd
import pickle

In [29]:
#top 2000개 단어
num_words = 2000
word_embeddings = pd.DataFrame(bamboo2vec.wv.syn0[:num_words, :], index=bamboo2vec.wv.index2word[:num_words])
word_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
<PAD>/Pad,-0.150759,-0.093838,0.042294,0.032093,-0.115384,0.012368,0.116375,0.007131,-0.170291,-0.153988,...,-0.115582,-0.123719,-0.193763,0.062940,0.067775,-0.058004,-0.068059,0.179474,0.097024,0.106724
./Punctuation,0.026793,-0.060600,0.085425,-0.145963,-0.118692,-0.036977,0.143720,-0.005639,-0.153097,-0.012977,...,-0.154144,-0.079539,-0.061695,0.023848,0.092723,-0.085759,-0.056398,0.166835,0.109141,0.006672
이/Josa,0.085074,0.021790,0.050082,-0.114358,-0.110462,-0.066833,0.153724,0.000110,-0.015940,-0.036423,...,0.039653,-0.042027,-0.106174,0.024525,0.174550,-0.045447,-0.104813,-0.052655,0.003032,0.029746
을/Josa,0.159536,-0.011369,0.209430,-0.121898,-0.062155,0.012108,0.225984,0.034409,0.055283,0.011856,...,-0.056312,0.033775,-0.166808,0.027437,0.114839,-0.056090,-0.075806,-0.096935,-0.183822,-0.127056
가/Josa,0.078237,0.016534,0.125696,-0.026570,-0.075922,-0.030022,0.011926,-0.058147,-0.040257,-0.034536,...,-0.039977,-0.018216,-0.152482,-0.068390,0.140578,-0.019761,-0.112848,-0.060661,0.002411,0.017626


In [30]:
%%time
USE_PREMADE_TSNE = False

tsne_filepath = 'tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(word_embeddings.values)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=word_embeddings.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

CPU times: user 48.3 s, sys: 4.47 s, total: 52.8 s
Wall time: 53.9 s


In [31]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [32]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='red', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);